In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/Users/chengpeng/Desktop/Research/STAT/CIBer')
import comonotonic as cm
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import copy
import utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import ensemble_ciber as ec
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTENC
import xlsxwriter

In [2]:
df = pd.read_csv('glass.csv')

In [3]:
colnames = [('X'+str(i)) for i in range(df.shape[1]-1)]
colnames.append('Y')
df.columns = colnames

In [4]:
categorical = []
cont_col = [i for i in range(df.shape[1]-1)]
discrete_feature_val = None

In [5]:
scaler = preprocessing.StandardScaler()
scale_col_name = ["X"+str(i) for i in cont_col]
df[scale_col_name] = scaler.fit_transform(df[scale_col_name])
reduced_df = utils.outlier_removal(df, cont_col)

In [6]:
x = reduced_df.iloc[:,:-1].to_numpy()
y = reduced_df.iloc[:,-1].to_numpy()
smote = SMOTE(random_state=0)
x_resample, y_resample = smote.fit_resample(x, y)

In [7]:
x_train,x_test,y_train,y_test = train_test_split(x_resample,y_resample,test_size=0.2)

In [8]:
c_como_demo = cm.clustered_comonotonic(x_train,y_train,discrete_feature_val,cont_col,
                                      categorical, 0.97, None, corrtype='mutual_info',
                                      discrete_method='mdlp')
c_como_demo.run()
c_como_predict = c_como_demo.predict(x_test)
print(classification_report(y_test,c_como_predict))

              precision    recall  f1-score   support

           1       0.64      0.75      0.69        12
           2       1.00      0.55      0.71        11
           3       0.65      0.73      0.69        15
           5       0.95      0.95      0.95        20
           6       1.00      1.00      1.00        13
           7       0.95      1.00      0.97        19

    accuracy                           0.86        90
   macro avg       0.86      0.83      0.84        90
weighted avg       0.87      0.86      0.85        90



In [9]:
c_como_demo.print_cluster()

[[0, 1, 6], [2], [3], [4], [5], [7], [8]]


In [17]:
cm.get_norm_mutual_info(x_train)

array([[1.        , 0.97185398, 0.87450609, 0.96490435, 0.9708512 ,
        0.81077654, 0.97476359, 0.36336161, 0.40931073],
       [0.97185398, 1.        , 0.86263287, 0.95304739, 0.95846074,
        0.79891348, 0.96245958, 0.35939157, 0.38991907],
       [0.87450609, 0.86263287, 1.        , 0.85361915, 0.8602662 ,
        0.73752905, 0.86322381, 0.23169984, 0.35775833],
       [0.96490435, 0.95304739, 0.85361915, 1.        , 0.95201945,
        0.79046621, 0.95606623, 0.36320023, 0.39169832],
       [0.9708512 , 0.95846074, 0.8602662 , 0.95201945, 1.        ,
        0.79638693, 0.96212448, 0.3475146 , 0.39083911],
       [0.81077654, 0.79891348, 0.73752905, 0.79046621, 0.79638693,
        1.        , 0.80004744, 0.29579308, 0.40330476],
       [0.97476359, 0.96245958, 0.86322381, 0.95606623, 0.96212448,
        0.80004744, 1.        , 0.35137679, 0.39903298],
       [0.36336161, 0.35939157, 0.23169984, 0.36320023, 0.3475146 ,
        0.29579308, 0.35137679, 1.        , 0.35723397],


In [18]:
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(x_train,y_train)
xgb_predict = xgb_clf.predict(x_test)
print(classification_report(y_test, xgb_predict))

              precision    recall  f1-score   support

           1       0.88      0.54      0.67        13
           2       0.85      0.89      0.87        19
           3       0.88      1.00      0.94        15
           5       0.94      1.00      0.97        16
           6       1.00      1.00      1.00        15
           7       0.92      1.00      0.96        12

    accuracy                           0.91        90
   macro avg       0.91      0.91      0.90        90
weighted avg       0.91      0.91      0.90        90



In [36]:
rf_clf = RandomForestClassifier()
rf_clf.fit(x_train,y_train)
rf_predict = rf_clf.predict(x_test)
print(classification_report(y_test, rf_predict))

/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           1       0.75      0.64      0.69        14
           2       0.71      0.77      0.74        13
           3       0.82      0.88      0.85        16
           5       1.00      1.00      1.00        18
           6       0.91      1.00      0.95        10
           7       1.00      0.95      0.97        19

    accuracy                           0.88        90
   macro avg       0.87      0.87      0.87        90
weighted avg       0.88      0.88      0.88        90



In [20]:
ada_clf = AdaBoostClassifier()
ada_clf.fit(x_train,y_train)
ada_predict = ada_clf.predict(x_test)
print(classification_report(y_test, ada_predict))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        13
           2       0.06      0.05      0.06        19
           3       0.34      1.00      0.51        15
           5       0.00      0.00      0.00        16
           6       1.00      1.00      1.00        15
           7       0.92      1.00      0.96        12

    accuracy                           0.48        90
   macro avg       0.39      0.51      0.42        90
weighted avg       0.36      0.48      0.39        90



/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(x_train, y_train)
lgb_predict = lgb_clf.predict(x_test)
lgb_predict = lgb_predict.round(0).astype('int')
print(classification_report(y_test, lgb_predict))

              precision    recall  f1-score   support

           1       0.91      0.77      0.83        13
           2       0.86      0.95      0.90        19
           3       0.93      0.93      0.93        15
           5       1.00      1.00      1.00        16
           6       1.00      1.00      1.00        15
           7       1.00      1.00      1.00        12

    accuracy                           0.94        90
   macro avg       0.95      0.94      0.94        90
weighted avg       0.95      0.94      0.94        90



In [22]:
DT_clf = tree.DecisionTreeClassifier()
DT_clf.fit(x_train,y_train)
DT_predict = DT_clf.predict(x_test)
print(classification_report(y_test, DT_predict))

              precision    recall  f1-score   support

           1       0.53      0.62      0.57        13
           2       0.86      0.63      0.73        19
           3       0.87      0.87      0.87        15
           5       0.94      1.00      0.97        16
           6       1.00      1.00      1.00        15
           7       0.86      1.00      0.92        12

    accuracy                           0.84        90
   macro avg       0.84      0.85      0.84        90
weighted avg       0.85      0.84      0.84        90



In [23]:
svm = SVC()
svm.fit(x_train, y_train)
svm_predict = svm.predict(x_test)
print(classification_report(y_test, svm_predict))

              precision    recall  f1-score   support

           1       0.27      1.00      0.43        13
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        15
           5       0.80      1.00      0.89        16
           6       0.92      0.80      0.86        15
           7       1.00      0.75      0.86        12

    accuracy                           0.56        90
   macro avg       0.50      0.59      0.50        90
weighted avg       0.47      0.56      0.48        90



/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being se

In [24]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_predict = lr.predict(x_test)
print(classification_report(y_test, lr_predict))

/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/r7user5/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

           1       0.30      0.46      0.36        13
           2       0.50      0.21      0.30        19
           3       0.56      0.67      0.61        15
           5       0.89      1.00      0.94        16
           6       1.00      0.93      0.97        15
           7       1.00      1.00      1.00        12

    accuracy                           0.69        90
   macro avg       0.71      0.71      0.70        90
weighted avg       0.70      0.69      0.68        90



In [24]:
ciber_nb = cm.clustered_comonotonic(x_train,y_train,discrete_feature_val,cont_col,
                                categorical, 1, None, corrtype='spearman',
                                discrete_method='mdlp')
ciber_nb.run()
ciber_nb_predict = ciber_nb.predict(x_test)
print(classification_report(y_test,ciber_nb_predict))

              precision    recall  f1-score   support

           1       0.64      0.64      0.64        14
           2       0.77      0.77      0.77        13
           3       0.69      0.69      0.69        16
           5       0.90      1.00      0.95        18
           6       1.00      1.00      1.00        10
           7       1.00      0.89      0.94        19

    accuracy                           0.83        90
   macro avg       0.83      0.83      0.83        90
weighted avg       0.84      0.83      0.83        90

